<a href="https://colab.research.google.com/github/nrnrk/public-resources/blob/main/google-colaboratory/wc-cuda/wc_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## `wc` コマンドの CUDA 実装

学習のために `wc` コマンドを CUDA で実装し直します。大きいファイルでは本家より早くなることをごく簡単な計測で確認できました。

### 準備

* Colaboratory で `編集` > `ノートブックの設定` で CUDA 対応のアクセラレータ(T4など)を選択
  * これにより関連のミドルウェアはインストールされます
* `PyCUDA` のインストール


In [3]:
# 2分くらいかかります
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660427 sha256=90f3c0a88905284576365bce8ff3fa55163420d6f1194e46e2c7d574b3d2c4a1
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


### .cubin の生成

`wc.cu` に CUDA カーネルを定義しています。実行速度を上げるために `.cubin` を事前に生成しておきます。

In [5]:
# T4 は sm_75 のアーキテクチャ
!nvcc -cubin -arch=sm_75 wc.cu -o wc.cubin

### wc.py を実行

`wc.cubin` をロードして、 `wc` コマンド相当のものを実行するコード `wc.py` を実行します。

In [9]:
!python wc.py wc.py

 79 208 2410 wc.py


### 時間の計測

`time` コマンドで簡易的に速度を計測する。テスト用の短いテキストと、長いテキストも事前に生成して、 `wc` コマンドと比較します。

In [10]:
# 短めのデータを生成
with open('short_text.txt', 'w') as f:
  for _ in range(100):
    f.write('short text!\n' * 5)

In [11]:
# とても長いデータを生成
with open('long_text.txt', 'w') as f:
  for _ in range(100000):
    f.write('super long text!\n' * 100)

In [12]:
!time wc short_text.txt

 500 1000 6000 short_text.txt

real	0m0.006s
user	0m0.000s
sys	0m0.001s


In [13]:
!time python wc.py short_text.txt

 500 1000 6000 short_text.txt

real	0m0.331s
user	0m0.236s
sys	0m0.203s


In [14]:
!time wc long_text.txt

 10000000  30000000 170000000 long_text.txt

real	0m1.814s
user	0m1.711s
sys	0m0.041s


In [15]:
!time python wc.py long_text.txt

 10000000 30000105 170000000 long_text.txt

real	0m0.478s
user	0m0.327s
sys	0m0.264s
